In [2]:
from openai import OpenAI
import pathlib
import textwrap
import os
import time
from tqdm.notebook import tqdm
from IPython.display import display
from IPython.display import Markdown
import json

In [3]:
client = OpenAI(
  organization='org-MiSTdq8wllMVuvQ1Es4GrHNX',
  project='proj_rO8d3QjByeL5WcL1nApaiy5B',
)

In [4]:
with open('./data/LinearLecNotes_Definitions.json', 'r') as f:
    data = json.load(f)

In [5]:
def get_json(content_id, prompt):
    sample = {"custom_id": f"{content_id}",
              "method": "POST",
              "url": "/v1/chat/completions",
              "body": {"model": "gpt-4o",
                       "messages": [{"role": "user", 
                                     "content": prompt}],
                       "max_tokens": 1500,
                       "temperature": 0.1,
                       "top_p":1,
                       "frequency_penalty":0,
                       "presence_penalty":0}}
    return sample

In [6]:
contents = []
for chapter, sections in (data.items()):
    for section, section_definitions in (sections.items()):
        i = 1;
        for q in (section_definitions):
            content_id = (chapter+"|"+section+"|"+str(i))
            i+=1
            message = fr"""Please convert the above definition into a conceptual true or false question with the answer and detailed explanation. The question must test the conceptual understanding of the concept presented. Your explanation must be concise but include an explanation of the main concepts used and use examples to illustrate these concepts. Use latex as required while only using the "$" delimiter. Do not ever create a newline in your output. Make sure the final output is in JSON format with three fields, "question", "answer" and "explanation". Definition: {q}"""
            contents.append(get_json(content_id, message))
            

In [307]:
with open('batches/batch_files/batch02_LinearTest_temp0-25.jsonl', 'w') as f:
    for record in contents:
        json_record = json.dumps(record)
        f.write(json_record + '\n')

In [308]:
from openai import OpenAI

batch_input_file = client.files.create(
  file=open("batches/batch_files/batch02_LinearTest_temp0-25.jsonl", "rb"),
  purpose="batch"
)

In [309]:
batch_input_file_id = batch_input_file.id
batch_input_file_id

'file-DFQOPFvM4fP36Q1rViyd57Fu'

In [310]:
batch = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "t/f questions - 102 - linear - w/content_id - fixed delimiters and newlines"
    }
)
batch_id = batch.id

In [315]:
batch = client.batches.retrieve(batch_id)
batch

Batch(id='batch_gLjmav6A70pbOCnEwnFOIHp2', completion_window='24h', created_at=1720547940, endpoint='/v1/chat/completions', input_file_id='file-DFQOPFvM4fP36Q1rViyd57Fu', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1720547963, error_file_id=None, errors=None, expired_at=None, expires_at=1720634340, failed_at=None, finalizing_at=1720547959, in_progress_at=1720547941, metadata={'description': 't/f questions - 102 - linear - w/content_id - fixed delimiters and newlines'}, output_file_id='file-nUHTaZaPHGNUXz54TexcwWus', request_counts=BatchRequestCounts(completed=102, failed=0, total=102))

In [316]:
output_file_id = batch.output_file_id
content = client.files.content(output_file_id)

In [317]:
with open("./batches/batch_responses/tf_linear_0temp.jsonl", "w") as f:
    f.write(content.content.decode("utf-8"))

In [348]:
questions = []
with open("./batches/batch_responses/tf_linear_0temp.jsonl", "r") as file:
    for line in file:
        json_object = json.loads(line)
        q = json_object['response']['body']['choices'][0]["message"]["content"]
        q = q.replace("`", "").replace("json", "").replace("\n", "")
        try:
            questions.append(json.loads(q))
        except:
            q = q.replace("\\", "\\\\")
            questions.append(json.loads(q))

In [349]:
formeted = (questions[3]['explanation'])
print(formeted)

A vector $\vec y$ is considered a linear combination of vectors $\vec v_1, \vec v_2, \dotsc, \vec v_p$ if it can be expressed in the form $\vec y = c_1 \vec v_1 + c_2 \vec v_2 + \cdots + c_p \vec v_p$, where $c_1, c_2, \dotsc, c_p$ are scalars. In this case, $\vec y = 3\vec v_1 - 2\vec v_2 + 0\vec v_3$ fits this definition with weights 3, -2, and 0. Therefore, the statement is true. For example, if $\vec v_1 = \begin{pmatrix}1 \\ 0\end{pmatrix}$, $\vec v_2 = \begin{pmatrix}0 \\ 1\end{pmatrix}$, and $\vec v_3 = \begin{pmatrix}1 \\ 1\end{pmatrix}$, then $\vec y = 3\begin{pmatrix}1 \\ 0\end{pmatrix} - 2\begin{pmatrix}0 \\ 1\end{pmatrix} + 0\begin{pmatrix}1 \\ 1\end{pmatrix} = \begin{pmatrix}3 \\ -2\end{pmatrix}$, which is a linear combination of $\vec v_1, \vec v_2, \vec v_3$.


In [350]:
formatted_questions = {}
fq = []
i = 0;
for q in tqdm(questions):
    i+=1;
    try:
        formatted_response = f"""
            \\textbf {{Question: {i}}} {q['question']}
            
            \\textbf{{Answer:}} {q['answer']}
    
            \\textbf{{Explanation:}} {q['explanation']}
            
            \\vspace{{0.5cm}} 
    
        """
    except:
        print(q);
    fq.append(formatted_response)

  0%|          | 0/102 [00:00<?, ?it/s]

In [351]:
fff = ''.join(fq[:])
f = open("problems/problemset.tex", "r")
p = f.read()
part = list(p.rpartition("\\vspace{1cm}"))
final = ''.join(part[0:2])
final = final + fff + part[2]
f = open("problems/102batch_linear_0temp.tex", "w")
f.write(final)
f.close()

In [345]:
(formatted_response)

'\n            \\textbf {Question: 102} True or False: If a symmetric matrix $A$ has both positive and negative eigenvalues, then the quadratic form $Q = \\vec{x}^{\\,T} A \\vec{x}$ is positive definite.\n            \n            \\textbf{Answer:} False\n    \n            \\textbf{Explanation:} A symmetric matrix $A$ with both positive and negative eigenvalues means that the quadratic form $Q = \\vec{x}^{\\,T} A \\vec{x}$ is indefinite. This is because a positive definite matrix requires all eigenvalues to be positive, and a negative definite matrix requires all eigenvalues to be negative. When there are both positive and negative eigenvalues, the quadratic form can take both positive and negative values, making it indefinite. For example, if $A$ has eigenvalues $\\lambda_1 = 2$ and $\\lambda_2 = -3$, then $Q$ can be positive for some vectors $\\vec{x}$ and negative for others.\n            \n            \\vspace{0.5cm} \n    \n        '